In [1]:
# 保存する前に消す！！！！
!huggingface-cli login --token 

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/imyu/.cache/huggingface/token
Login successful


In [2]:
!huggingface-cli whoami

imyu


In [3]:
# Load model directly
# 自分の
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B")

/opt/anaconda3/envs/bias/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
model.safetensors.index.json: 100%|██████████| 23.9k/23.9k [00:00<00:00, 10.4MB/s]
model-00001-of-00004.safetensors: 100%|██████████| 4.98G/4.98G [01:49<00:00, 45.6MB/s]
model-00002-of-00004.safetensors: 100%|██████████| 5.00G/5.00G [01:52<00:00, 44.3MB/s]
model-00003-of-00004.safetensors: 100%|██████████| 4.92G/4.92G [01:49<00:00, 44.9MB/s]
model-00004-of-00004.safetensors: 100%|██████████| 1.17G/1.17G [00:25<00:00, 45.1MB/s]
generation_config.json: 100%|██████████| 177/177 [00:00<00:00, 93.3kB/s]


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

/opt/anaconda3/envs/bias/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
generation_config.json: 100%|██████████| 187/187 [00:00<00:00, 70.2kB/s]


In [4]:
prompt = "Hey, are you conscious? Can you talk to me?"
inputs = tokenizer(prompt, return_tensors="pt")
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


KeyboardInterrupt: 

In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B"

pipeline = transformers.pipeline("text-generation", model=model_id, model_kwargs={"torch_dtype": torch.bfloat16}, device_map="auto")
pipeline("Hey how are you doing today?")